In [ ]:
pip install selenium

In [ ]:
pip install webdriver-manager

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoAlertPresentException
from selenium.webdriver.common.action_chains import ActionChains

import time

# 🔽 사용자 입력 부분
site_prefix = "sstt"   # <-- ssct 대신 다른 값도 가능
user_ids = ["sstt900514","sstt960718","sstt930923"]
login_pw = "1234"
new_pw = "123qwe!@#"

# 🔽 최종 사이트 URL
url = f"https://{site_prefix}.edup.co.kr/"

for login_id in user_ids:
    print(f"=== {login_id} 로그인 시작 ===")
    login_pw = "1234"
    new_pw = "123qwe!@#"

    def safe_accept_alert(driver):
        """alert이 있으면 accept, 없으면 그냥 넘어감"""
        try:
            alert = driver.switch_to.alert
            print("팝업 내용:", alert.text)
            alert.accept()
            time.sleep(0.5)
        except NoAlertPresentException:
            pass

    # 브라우저 실행
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    driver.maximize_window()
    time.sleep(2)

    # 아이디 입력
    id_input = driver.find_element(By.ID, "login_id")
    id_input.send_keys(login_id)

    # 비밀번호 입력
    pw_input = driver.find_element(By.ID, "login_pw")
    pw_input.send_keys(login_pw)

    # 로그인 버튼 클릭
    driver.find_element(By.ID, "smt_login").click()

    time.sleep(2) 



    # 1) alert 대화상자 처리
    try:
        driver.switch_to.alert.accept()
    except:
        pass
    time.sleep(0.6)

    # 1) alert 대화상자 처리
    try:
        driver.switch_to.alert.accept()
    except:
        pass

    # # 2) 모달 팝업 처리
    # try:
    #     driver.find_element(By.CSS_SELECTOR, ".btn_confirm").click()
    # except:
    #     pass

    time.sleep(0.6)


    login_id_box = driver.find_elements(By.ID, "login_id")

    if login_id_box:   # 요소가 존재할 경우만
        login_id_box = login_id_box[0]
        login_id_box.click()
        login_id_box.clear()
        login_id_box.send_keys(Keys.CONTROL, 'a')
        login_id_box.send_keys(Keys.DELETE)
        login_id_box.send_keys(login_id)

    # login_pw 입력창이 존재하는지 확인
    login_pw_box = driver.find_elements(By.ID, "login_pw")

    if login_pw_box:
        login_pw_box = login_pw_box[0]
        login_pw_box.click()
        login_pw_box.clear()
        login_pw_box.send_keys(Keys.CONTROL, 'a')
        login_pw_box.send_keys(Keys.DELETE)
        login_pw_box.send_keys(new_pw)

        time.sleep(0.5)

        # 로그인 버튼 클릭
    login_btn = driver.find_elements(By.ID, "smt_login")
    if login_btn:
            login_btn[0].click()
    time.sleep(1)
    safe_accept_alert(driver) 



    try:
        # 🔍 1) 비밀번호 변경 페이지가 있는지 확인
        pw_input = driver.find_element(By.NAME, "mb_password")
        pw_confirm_input = driver.find_element(By.NAME, "mb_password_confirm")

        # 👉 비밀번호 입력
        pw_input.send_keys(new_pw)
        pw_confirm_input.send_keys(new_pw)

        # 🔘 비밀번호 변경 버튼 클릭
        driver.find_element(By.CSS_SELECTOR, "input.btn_submit").click()
        time.sleep(1)

        # ⏩ 변경 완료 후 '내 강의실' 버튼 클릭
        driver.find_element(By.CSS_SELECTOR, "a.btn_myroom").click()

    except NoSuchElementException:
        # 🟢 비밀번호 변경 페이지가 없으면 바로 내 강의실로
        driver.find_element(By.CSS_SELECTOR, "a.btn_myroom").click()
        
        

    # ------------------- 내 강의실 이동 -------------------
    safe_accept_alert(driver)

    try:
        myroom_btn = driver.find_element(By.CSS_SELECTOR, "a.btn_myroom")
        myroom_btn.click()
    except NoSuchElementException:
        print("내 강의실 버튼이 존재하지 않습니다.")
        
    time.sleep(1)  # 모달/iframe 로딩 대기

    # 1) iframe 체크
    iframes = driver.find_elements(By.TAG_NAME, "iframe")
    frame_switched = False
    for frame in iframes:
        try:
            driver.switch_to.frame(frame)
            # 버튼 있는지 확인
            if driver.find_elements(By.ID, "substituteClass_no"):
                frame_switched = True
                print("iframe 내부로 전환")
                break
            driver.switch_to.default_content()
        except:
            continue

    # 2) 버튼 클릭 시도
    try:
        # 동의 버튼이 나타날 때까지 최대 10초 기다림
        agree_btn = WebDriverWait(driver, 0.6).until(
            EC.element_to_be_clickable((By.ID, "substituteClass_no"))
        )
        # JS click 사용
        driver.execute_script("arguments[0].click();", agree_btn)
        print("동의 버튼 클릭 완료")
        
        try:
            WebDriverWait(driver, 1).until(EC.alert_is_present())
            alert = driver.switch_to.alert
            print("알림 텍스트:", alert.text)
            alert.accept()
            print("알림 창 닫음")
        except TimeoutException:
            print("알림 창이 안 떴습니다.")
        
    except TimeoutException:
        print("동의 버튼이 나타나지 않음, 넘어갑니다.")
    time.sleep(1)
        
        # ---------------- 기수 선택 ----------------
    dropdown = WebDriverWait(driver, 0.6).until(
        EC.element_to_be_clickable((By.ID, "p_id_select"))
    )
    dropdown.click()
    time.sleep(0.5)

    # 첫 번째 기수 선택
    first_item = driver.find_element(By.CSS_SELECTOR, "#p_id_select .select_list ul li")
    first_item.click()
    print("첫 번째 기수 선택 완료")
    time.sleep(0.5)

    # ---------------- 과정 선택 ----------------
    course_dropdown = WebDriverWait(driver, 0.6).until(
        EC.element_to_be_clickable((By.ID, "s_id_select"))
    )
    course_dropdown.click()
    time.sleep(0.5)

    course_items = driver.find_elements(By.CSS_SELECTOR, "#s_id_select .select_list ul li")
    main_window = driver.current_window_handle

    for i in range(len(course_items)):
        # 과정 요소를 매번 새로 찾기
        course_items = driver.find_elements(By.CSS_SELECTOR, "#s_id_select .select_list ul li")
        course = course_items[i]

        # 15번째는 스크롤 필요
        if i == 14:  # 15번째
            driver.execute_script("arguments[0].scrollIntoView(true);", course)
            time.sleep(0.5)
        driver.execute_script("arguments[0].click();", course)
        
        # 과정 클릭
        driver.execute_script("arguments[0].click();", course)
        print(f"{i+1}번째 과정 선택 완료")
        time.sleep(0.6)

        # 강의보기 버튼 클릭 (항상 단 하나)
        lec_btn = WebDriverWait(driver, 0.6).until(
        EC.element_to_be_clickable((By.XPATH, "//a[contains(@href,'go_lecview4')]"))
        )
        driver.execute_script("arguments[0].click();", lec_btn)
        print(f"{i+1}번째 과정 강의보기 클릭")

        # 새 창으로 전환
        time.sleep(0.6)  # 새 창이 뜨도록 대기
        all_windows = driver.window_handles
        for handle in all_windows:
            if handle != main_window:
                driver.switch_to.window(handle)
                print(f"{i+1}번째 과정 새 창 전환")
                time.sleep(5)  # 5초 대기
                driver.close()
                print(f"{i+1}번째 과정 새 창 닫기")
                break

        # 다시 원래 창으로 복귀
        driver.switch_to.window(main_window)
        time.sleep(0.3)
    try:
        logout_btn = driver.find_element(By.CSS_SELECTOR, "a[href$='logout.php']")
        driver.execute_script("arguments[0].click();", logout_btn)
        safe_accept_alert(driver)
        driver.quit()
        
    except:
        print(f"{login_id} 로그아웃 실패, 강제 복귀")
        driver.switch_to.window(main_window)
        
print("모든 아이디 처리 완료")
driver.quit()

=== sstt900514 로그인 시작 ===
팝업 내용: 이미 다른장소에서 로그인되어 있습니다.

해당 아이디를 모두 로그아웃 합니다.
내 강의실 버튼이 존재하지 않습니다.
동의 버튼이 나타나지 않음, 넘어갑니다.
첫 번째 기수 선택 완료
1번째 과정 선택 완료
1번째 과정 강의보기 클릭
1번째 과정 새 창 전환
1번째 과정 새 창 닫기
2번째 과정 선택 완료
2번째 과정 강의보기 클릭
2번째 과정 새 창 전환
2번째 과정 새 창 닫기
3번째 과정 선택 완료
3번째 과정 강의보기 클릭
3번째 과정 새 창 전환
3번째 과정 새 창 닫기
4번째 과정 선택 완료
4번째 과정 강의보기 클릭
4번째 과정 새 창 전환
4번째 과정 새 창 닫기
5번째 과정 선택 완료
5번째 과정 강의보기 클릭
5번째 과정 새 창 전환
5번째 과정 새 창 닫기
6번째 과정 선택 완료
6번째 과정 강의보기 클릭
6번째 과정 새 창 전환
6번째 과정 새 창 닫기
7번째 과정 선택 완료
7번째 과정 강의보기 클릭
7번째 과정 새 창 전환
7번째 과정 새 창 닫기
8번째 과정 선택 완료
8번째 과정 강의보기 클릭
8번째 과정 새 창 전환
8번째 과정 새 창 닫기
9번째 과정 선택 완료
9번째 과정 강의보기 클릭
9번째 과정 새 창 전환
9번째 과정 새 창 닫기
10번째 과정 선택 완료
10번째 과정 강의보기 클릭
10번째 과정 새 창 전환
10번째 과정 새 창 닫기
11번째 과정 선택 완료
11번째 과정 강의보기 클릭
11번째 과정 새 창 전환
11번째 과정 새 창 닫기
12번째 과정 선택 완료
12번째 과정 강의보기 클릭
12번째 과정 새 창 전환
12번째 과정 새 창 닫기
13번째 과정 선택 완료
13번째 과정 강의보기 클릭
13번째 과정 새 창 전환
13번째 과정 새 창 닫기
14번째 과정 선택 완료
14번째 과정 강의보기 클릭
14번째 과정 새 창 전환
14번째 과정 새 창 닫기
15번째 과정 선택 완료
15번째 과정 강의보기 클릭
15번째 과정 새 창 전환
15번째 과정 새 창 닫기
=